# Imports

In [ ]:
!pip install facenet-pytorch --force-reinstall --no-cache-dir

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import pickle
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cosine
from tqdm import tqdm

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
# load dictionary of {filename: embedding_vector}
with open("embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)

# EDA

In [ ]:
# access one entry
print(len(embeddings))                # number of samples
print(list(embeddings.keys())[:5])    # filenames
vec = embeddings["00001.jpg"]        # numpy array shape (512,)

In [ ]:
# convert to torch tensor when used
emb_target = torch.tensor(vec).unsqueeze(0).cuda()  # shape [1,512]
print(emb_target.shape)

In [ ]:
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)
transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor()
])

In [ ]:
img_00001 = Image.open("00001.jpg").convert("RGB")
x_00001 = transform(img_00001).unsqueeze(0).to(device)
img_00002 = Image.open("00002.jpg").convert("RGB")
x_00002 = transform(img_00002).unsqueeze(0).to(device)

In [ ]:
emb_00001 = model(x_00001*2-1).detach().cpu().numpy()[0]
emb_00002 = model(x_00002*2-1).detach().cpu().numpy()[0]

In [ ]:
img_00001

In [ ]:
img_00002

In [ ]:
img_00001.size, x_00001.shape

In [ ]:
random_data = np.random.randint(
    0, 256,
    size=(256, 256, 3),
    dtype=np.uint8
)
random_img = Image.fromarray(random_data, 'RGB')

In [ ]:
random_img

In [ ]:
random_image_tensor = transform(random_img).unsqueeze(0).to(device)

In [ ]:
random_image_emb = model(random_image_tensor*2-1).detach().cpu().numpy()[0]

In [ ]:
random_image_emb.shape, emb_00001.shape

In [ ]:
cosine_dist = cosine(random_image_emb, emb_00001)
cosine_dist

Because the random image has no relationship with the face embedding, the distance is close to 1

In [ ]:
cosine_dist = cosine(emb_00001, emb_00002)
cosine_dist

Because the 2 face embeddings are intentionally different, their distance is closer to 2 which means closer to opposite.

# General functions

In [ ]:
def get_initial_image_tensor_uniform(size=160):
    tensor = torch.rand(1, 3, size, size, device=device)
    tensor.requires_grad_(True)
    return tensor

In [ ]:
def get_initial_image_tensor_gaussian(size=160):
    tensor = torch.randn(1, 3, size, size, device=device)
    tensor.requires_grad_(True)
    return tensor

In [ ]:
def get_initial_image_tensor_constant_and_gaussian(size=160, color_value=0.0):
    tensor = torch.full((1, 3, size, size), fill_value=color_value, device=device)
    # Add a tiny bit of noise to break symmetry (helps gradients start moving)
    tensor = tensor + (torch.randn_like(tensor) * 0.01)
    tensor.requires_grad_(True)
    return tensor

In [ ]:
def display_loss_graph(loss_list, log_scale=False):
    plt.plot(loss_list)
    if log_scale:
        plt.yscale('log')
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.title("Loss per Step")
    plt.show()

In [ ]:
def save_and_display_image(image, filename):
    final_image = torch.tanh(image.detach().cpu().squeeze(0))
    final_image = (final_image * 0.5) + 0.5

    final_image = transforms.ToPILImage()(final_image)
    final_image.save(filename)
    display(final_image)

In [ ]:
target_embedding = torch.tensor(emb_00001, dtype=torch.float32).unsqueeze(0).to(device)

### Attempt 1: uniform initialization

In [ ]:
image = get_initial_image_tensor_uniform()
num_steps = 200
optimizer = optim.Adam([image], lr=0.01)
loss_fn = nn.MSELoss()

In [ ]:
loss_list = []
for i in tqdm(range(num_steps)):
    optimizer.zero_grad()

    normalized_image = torch.tanh(image * 2 - 1)
    current_embedding = model(normalized_image)
    loss = loss_fn(current_embedding, target_embedding)

    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{num_steps}], Loss: {loss.item():.6f}")

    loss_list.append(loss.item())

final_image = image.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)

In [ ]:
display_loss_graph(loss_list=loss_list)

In [ ]:
save_and_display_image(final_image, "exp1_uniform_init.png")

In [ ]:
cosine_dist = cosine(emb_00001, final_embedding)
cosine_dist

### Attempt 2: gaussian initialization

In [ ]:
image = get_initial_image_tensor_gaussian()
num_steps = 200
optimizer = optim.Adam([image], lr=0.01)
loss_fn = nn.MSELoss()

In [ ]:
loss_list = []
for i in tqdm(range(num_steps)):
    optimizer.zero_grad()

    normalized_image = torch.tanh(image * 2 - 1)
    current_embedding = model(normalized_image)
    loss = loss_fn(current_embedding, target_embedding)

    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{num_steps}], Loss: {loss.item():.6f}")

    loss_list.append(loss.item())

final_image = image.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)

In [ ]:
display_loss_graph(loss_list=loss_list)

In [ ]:
save_and_display_image(final_image, "exp2_gaussian_init.png")

In [ ]:
cosine_dist = cosine(emb_00001, final_embedding)
cosine_dist

### Attempt 3: constant initialization and a bit of gaussian noise

In [ ]:
image = get_initial_image_tensor_constant_and_gaussian()
num_steps = 200
optimizer = optim.Adam([image], lr=0.01)
loss_fn = nn.MSELoss()

In [ ]:
loss_list = []
for i in tqdm(range(num_steps)):
    optimizer.zero_grad()

    normalized_image_input = torch.tanh(image * 2 - 1)
    current_embedding = model(normalized_image_input)
    loss = loss_fn(current_embedding, target_embedding)
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{num_steps}], Loss: {loss.item():.6f}")

    loss_list.append(loss.item())

final_image = image.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)

In [ ]:
display_loss_graph(loss_list=loss_list)

In [ ]:
save_and_display_image(final_image, "exp3_small_gaussian_init.png")

In [ ]:
cosine_dist = cosine(emb_00001, final_embedding)
cosine_dist

### Attempt 4: Adding Total Variation to the loss function

In [ ]:
def get_tv_loss(img_tensor):
    """
    Computes Total Variation Loss.
    Expected input shape: (Batch, Channels, Height, Width)
    """
    # Calculate horizontal differences (between columns)
    # Select all columns except the last one, minus all columns except the first one
    diff_h = img_tensor[:, :, :, :-1] - img_tensor[:, :, :, 1:]

    # Calculate vertical differences (between rows)
    # Select all rows except the last one, minus all rows except the first one
    diff_w = img_tensor[:, :, :-1, :] - img_tensor[:, :, 1:, :]

    # Sum the absolute differences
    tv_loss = torch.sum(torch.abs(diff_h)) + torch.sum(torch.abs(diff_w))

    return tv_loss

In [ ]:
image = get_initial_image_tensor_constant_and_gaussian()
num_steps = 1000
tv_weight = 4e-7
optimizer = optim.Adam([image], lr=0.01)
loss_fn = nn.MSELoss()

In [ ]:
loss_list = []

for i in tqdm(range(num_steps)):
    # Clear old gradients
    optimizer.zero_grad()

    normalized_image_input = torch.tanh(image * 2 - 1)
    current_embedding = model(normalized_image_input)

    loss_mse = loss_fn(current_embedding, target_embedding)
    loss_tv = get_tv_loss(image)
    total_loss = loss_mse + (tv_weight * loss_tv)

    total_loss.backward()

    optimizer.step()

    if (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{num_steps}], Loss: {total_loss.item():.6f}")

    loss_list.append(total_loss.item())

final_image = image.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)

In [ ]:
display_loss_graph(loss_list=loss_list)

In [ ]:
save_and_display_image(final_image, "exp4_small_gausian_init_and_tv_weight_4e-07.png")

In [ ]:
cosine_dist = cosine(emb_00001, final_embedding)
cosine_dist

### Attempt 5: gaussian initialization with tv loss

In [ ]:
image = get_initial_image_tensor_gaussian()
num_steps = 1000
tv_weight = 1e-6
optimizer = optim.Adam([image], lr=0.01)
loss_fn = nn.MSELoss()

In [ ]:
loss_list = []

for i in tqdm(range(num_steps)):
    # Clear old gradients
    optimizer.zero_grad()

    normalized_image_input = torch.tanh(image * 2 - 1)
    current_embedding = model(normalized_image_input)

    loss_mse = loss_fn(current_embedding, target_embedding)
    loss_tv = get_tv_loss(image)
    total_loss = loss_mse + (tv_weight * loss_tv)

    total_loss.backward()

    optimizer.step()

    if (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{num_steps}], Loss: {total_loss.item():.6f}")

    loss_list.append(total_loss.item())

final_image = image.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)

In [ ]:
display_loss_graph(loss_list=loss_list)

In [ ]:
save_and_display_image(final_image, "exp5_gaussian_init_and_tv_weight_1e-06.png")

In [ ]:
cosine_dist = cosine(emb_00001, final_embedding)
cosine_dist

### Attempt 6: man face initialization

In [ ]:
image_pil = Image.open("00000.jpg").convert("RGB")
image = transform(image_pil).unsqueeze(0).to(device).requires_grad_(True)

In [ ]:
image_pil

In [ ]:
target_embedding = torch.tensor(emb_00002, dtype=torch.float32).unsqueeze(0).to(device)

In [ ]:
num_steps = 1000
tv_weight = 1e-7
optimizer = optim.Adam([image], lr=0.01)
loss_fn = nn.MSELoss()

In [ ]:
loss_list = []

for i in tqdm(range(num_steps)):
    # Clear old gradients
    optimizer.zero_grad()

    normalized_image_input = torch.tanh(image * 2 - 1)
    current_embedding = model(normalized_image_input)

    loss_mse = loss_fn(current_embedding, target_embedding)
    loss_tv = get_tv_loss(image)
    total_loss = loss_mse + (tv_weight * loss_tv)

    total_loss.backward()

    optimizer.step()

    if (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{num_steps}], Loss: {total_loss.item():.6f}")

    loss_list.append(total_loss.item())

final_image = image.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)

In [ ]:
display_loss_graph(loss_list=loss_list)

In [ ]:
save_and_display_image(final_image, "exp6_man_face_init_and_tv_weight_1e-07_b.png")

In [ ]:
cosine_dist = cosine(emb_00002, final_embedding)
cosine_dist

### Attempt 7 - Input Augmentation

In [ ]:
import torchvision.transforms as T

# Define a jitter transform
jitter = T.Compose([
    T.RandomAffine(degrees=0, translate=(0.05, 0.05))
])
loss_list = []

for i in tqdm(range(num_steps)):
    # Clear old gradients
    optimizer.zero_grad()
    jittered_image = jitter(image)
    normalized_image_input = torch.tanh(jittered_image * 2 - 1)
    current_embedding = model(normalized_image_input)

    loss_mse = loss_fn(current_embedding, target_embedding)
    loss_tv = get_tv_loss(image)
    total_loss = loss_mse + (tv_weight * loss_tv)

    total_loss.backward()

    optimizer.step()

    if (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{num_steps}], Loss: {total_loss.item():.6f}")

    loss_list.append(total_loss.item())

final_image = image.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)

In [ ]:
save_and_display_image(final_image, "exp7_man_face_init_and_tv_weight_1e-07_b.png")

In [ ]:
cosine_dist = cosine(emb_00002, final_embedding)
cosine_dist

Attempt 8 - cosine similarity loss

In [ ]:
import torchvision.transforms as T

# Define a jitter transform
jitter = T.Compose([
    T.RandomAffine(degrees=0, translate=(0.05, 0.05)) # Shift image slightly
])
loss_list = []

for i in tqdm(range(num_steps)):
    # Clear old gradients
    optimizer.zero_grad()
    normalized_image_input = torch.tanh(image * 2 - 1)
    current_embedding = model(normalized_image_input)

    cosine_loss = 1 - torch.nn.functional.cosine_similarity(current_embedding, target_embedding, dim=1).mean()
    loss_tv = get_tv_loss(image)
    total_loss = cosine_loss + (tv_weight * loss_tv)

    total_loss.backward()

    optimizer.step()

    if (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{num_steps}], Loss: {total_loss.item():.6f}")

    loss_list.append(total_loss.item())

final_image = image.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)

In [ ]:
save_and_display_image(final_image, "exp8_man_face_init_and_tv_weight_1e-07_b_and_cosine_loss.png")